In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../backup/operating_points.csv', sep=';')
df.head()

,Geo Point,Geo Shape,PTCAR ID,Code TAF/TAP,Nom symbolique,Abréviation LST FR courte,Abréviation LST NL courte,Abréviation LST FR complète,Abréviation LST NL complète,Nom FR court,Nom NL court,Nom FR moyen,Nom NL moyen,Nom FR complet,Nom NL complet,Classification NL,Classification EN,Classification FR
0,"50.923363496226415, 5.497283037498502","{""coordinates"": [5.497283037498502, 50.9233634...",443,BE00443,FKGLF,GENK-ZD-FORD,GENK-ZD-FORD,GENK-ZUID-L.O.-FORD,GENK-ZUID-L.O.-FORD,Genk-Zd-Ford,Genk-Zd-Ford,Genk-Zuid-L.O.-Ford,Genk-Zuid-L.O.-Ford,Genk-Zuid-L.O.-Ford,Genk-Zuid-L.O.-Ford,Station,Station,Gare
1,"51.30588400152173, 4.294961995341565","{""coordinates"": [4.294961995341565, 51.3058840...",1747,BE01747,VEVKA,V.EVONIK,V.EVONIK,VERB.EVONIK DEGUSSA ANTWERPEN,VERB.EVONIK DEGUSSA ANTWERPEN,V.Evonik,V.Evonik,Verb.Evonik Degussa,Verb.Evonik Degussa,Verb.Evonik Degussa Antwerpen,Verb.Evonik Degussa Antwerpen,Verbinding,Connection,Raccordement
2,"50.40057400186748, 4.458826995197694","{""coordinates"": [4.458826995197694, 50.4005740...",275,BE00275,GCRA,CHARLEROI-AT,CHARLEROI-AT,CHARLEROI-A.T.,CHARLEROI-A.T.,Charleroi-AT,Charleroi-AT,Charleroi-A.T.,Charleroi-A.T.,Charleroi-A.T.,Charleroi-A.T.,Dienstinstallatie,Service installation,Installation de service
3,"50.41133112791496, 4.522441728210868","{""coordinates"": [4.522441728210868, 50.4113311...",1978,BE01978,FCLGS,CHATELET-GAS,CHATELET-GAS,CHATELET-GASOIL,CHATELET-GASOIL,Châtelet-Gas,Châtelet-Gas,Châtelet-Gasoil,Châtelet-Gasoil,Châtelet-Gasoil,Châtelet-Gasoil,Dienstinstallatie,Service installation,Installation de service
4,"50.45350230747429, 4.822607107568604","{""coordinates"": [4.822607107568604, 50.4535023...",1010,BE01010,FEOPE,RONET-PE,RONET-PE,RONET-P.E.,RONET-P.E.,Ronet-PE,Ronet-PE,Ronet-P.E.,Ronet-P.E.,Ronet-P.E.,Ronet-P.E.,Dienstinstallatie,Service installation,Installation de service


In [28]:
print(df['Classification FR'].unique())

# Create a dict with colors for each classification
colors = [
    '#ff0000', # Station
    '#ff8000', # Connection
    '#ffff00', # Service installation
    '#80ff00', # 6
    '#00ff00', # Stop in open track
    '#00ff80', # Net borde
    '#00ffff', # Service stop
    '#0080ff', # Grid
    '#0000ff', # Junction
    '#8000ff', # Movable bridge
    '#ff00ff', # Other
    '#ff0080', # 8
]
classifications = ['Station', 'Connection', 'Service installation', '6', 'Stop in open track',
 'Net borde', 'Service stop', 'Grid', 'Junction', 'Movable bridge', 'Other', '8']

color_dict = dict()
for i in range(len(classifications)):
    color_dict[classifications[i]] = colors[i]

print(color_dict)

# Plot the points on the Belgium map, different color for each classification
import folium

# Create a map, centered on Belgium, don't allow to zoom
m = folium.Map(
    location=[50.503887, 4.469936],
    # Easily readable tiles
    tiles='cartodbpositron',
    zoom_start=8,
    zoom_control=False,
    scrollWheelZoom=False,
    dragging=False
)

for index, row in df.iterrows():
    # Get the classification
    classification = row['Classification EN']
    # Get the coordinates
    lat, lon = map(float, row['Geo Point'].split(','))
    # Add the marker, with a color depending on the classification
    if classification in [
        "Station",
        "Stop in open track",
        # "Service stop",
    ]:
        folium.CircleMarker(
            location=[lat, lon],
            radius=1,
            color=color_dict[classification],
            fill_color=color_dict[classification]
        ).add_to(m)

# Display the map
m

['Gare' 'Raccordement' 'Installation de service' '6' "Point d'arrêt"
 'Signal de bloc avec repérage' "Point d'arrêt de service" 'Gril'
 'Bifurcation' 'Pont mobile' 'Autre' 'Frontière du réseau']
{'Station': '#ff0000', 'Connection': '#ff8000', 'Service installation': '#ffff00', '6': '#80ff00', 'Stop in open track': '#00ff00', 'Net borde': '#00ff80', 'Service stop': '#00ffff', 'Grid': '#0080ff', 'Junction': '#0000ff', 'Movable bridge': '#8000ff', 'Other': '#ff00ff', '8': '#ff0080'}


In [1]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [ ]:

# Set up the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
open_meteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

coordinates = {
    "Hasselt": [50.930965, 5.338333],
    "Brussels": [50.85045, 4.34878],
    "Antwerp": [51.21989, 4.40346],
    "Ghent": [51.05, 3.71667],
    "Charleroi": [50.41136, 4.44448],
}

for city, coordinate in coordinates.items():
    params = {
        # Brussels coordinates
        "latitude": coordinate[0],
        "longitude": coordinate[1],
        "hourly": ["temperature_2m", "relative_humidity_2m"],
        "start_date": "2022-08-22",
        "end_date": "2023-09-13"
    }
    responses = open_meteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s"),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly_temperature_2m,
        "relative_humidity": hourly_relative_humidity_2m
    }

    hourly_dataframe = pd.DataFrame(data=hourly_data)
    print(hourly_dataframe)

    # Create a dataframe with hourly data and save it to a csv file
    hourly_dataframe.to_csv(f"./assets/weather/hourly_data_{city.lower()}.csv", index=False)
